**PART - 1**

In [35]:
from collections import deque
from queue import PriorityQueue

Reading File

In [16]:
def open_File(filename):
    matrix = []
    with open(filename, 'r') as f:
        for line in f:
            row = line.strip().split()
            matrix.append(row)
    return matrix

matrix = open_File("file.txt")
matrix

[['10S0100'],
 ['1100011'],
 ['0101000'],
 ['1101101'],
 ['0101000'],
 ['0111011'],
 ['G000000']]

BFS

In [26]:
from collections import deque

def BFS(matrix):
    def Valid(x, y):
        return 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] != '1' and not visited[x][y]

    start = (0, 0)
    queue = deque([(start, [start])])
    visited = [[False for _ in range(len(matrix[0]))] for _ in range(len(matrix))]

    while queue:
        (x, y), path = queue.popleft()
        if matrix[x][y] == 'G':  # Check if the current cell is the goal
            return path

        visited[x][y] = True
        for direction_x, direction_y in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
            next_x, next_y = x + direction_x, y + direction_y
            if Valid(next_x, next_y):
                new_path = path + [(next_x, next_y)]
                queue.append(((next_x, next_y), new_path))
                visited[next_x][next_y] = True

    return -1

In [27]:
Path = BFS(matrix)
if Path == -1:
  print("No Path Exists")
else:
  print("BFS Path:", Path)

BFS Path: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4), (5, 4), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)]


DFS

In [29]:
def DFS(matrix):
    def Valid(x, y):
        return 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] != '1' and not visited[x][y]

    start = (0, 0)
    stack = [(start, [start])]
    visited = [[False for _ in range(len(matrix[0]))] for _ in range(len(matrix))]

    while stack:
        (x, y), path = stack.pop()
        if matrix[x][y] == 'G':  # Check if the current cell is the goal
            return path

        visited[x][y] = True
        for direction_x, direction_y in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
            next_x, next_y = x + direction_x, y + direction_y
            if Valid(next_x, next_y):
                new_path = path + [(next_x, next_y)]
                stack.append(((next_x, next_y), new_path))
                visited[next_x][next_y] = True

    return -1

In [31]:
Path = DFS(matrix)
if Path == -1:
  print("No Path Exists")
else:
  print("DFS Path:", Path)

DFS Path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4), (5, 4), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)]


**PART - 2**

In [47]:
def Uniform_Cost(graph, start, target):
    visited = set()
    queue = PriorityQueue()
    queue.put((0, start))
    parent = {}
    node_cost = {start: 0}

    while not queue.empty():
        current_cost, node = queue.get()
        if node not in visited:
            visited.add(node)

            if node == target:
                break

            for neighbor, weight in graph.get(node, []):
                new_cost = node_cost[node] + weight
                if neighbor not in node_cost or new_cost < node_cost[neighbor]:
                    node_cost[neighbor] = new_cost
                    priority = new_cost
                    queue.put((priority, neighbor))
                    parent[neighbor] = node

    if target not in parent:
        return None, float('inf')

    path = []
    curr_node = target
    while curr_node != start:
        path.append(curr_node)
        curr_node = parent[curr_node]
    path.append(start)
    path.reverse()

    return path, node_cost[target]

In [50]:
graph = {
    'S': [('A', 5), ('B', 2)],
    'A': [('B', 15), ('G', 8)],
    'B': [('S', 1)],
    'G':[]
}

start_node = 'S'
target_node = 'G'
path, cost = Uniform_Cost(graph, start_node, target_node)

if path:
    print("Shortest Path:", path)
    print("Total cost:", cost)
else:
    print("No path found.")

Shortest Path: ['S', 'A', 'G']
Total cost: 13


**PART - 3**

In [51]:
matrix = open_File("file1.txt")
matrix

[['10S0100'],
 ['1100011'],
 ['0101+00'],
 ['11+11+1'],
 ['010+0+0'],
 ['0111011'],
 ['G000000']]

In [58]:
def heuristic_func(node, target):
    return abs(node[0] - target[0]) + abs(node[1] - target[1])

def neighbors(node, matrix):
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    neighbors = []
    for dx, dy in directions:
        x, y = node[0] + dx, node[1] + dy
        while 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] != '1':
            if matrix[x][y] != '+':
                neighbors.append((x, y))
            x += dx
            y += dy
    return neighbors

def a_star(matrix, start, target):
    open_list = [(0, start)]
    parent = {}
    cost_so_far = {start: 0}

    while open_list:
        _, current = min(open_list)
        open_list.remove((_, current))

        if current == target:
            break

        for next_node in neighbors(current, matrix):
            new_cost = cost_so_far[current] + (matrix[next_node[0]][next_node[1]] == '+')
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic_func(next_node, target)
                open_list.append((priority, next_node))
                parent[next_node] = current

    path = []
    curr_node = target
    while curr_node in parent:
        path.append(curr_node)
        current = parent[curr_node]
    path.append(start)
    path.reverse()

    return path

In [60]:
start = (0, 2)
target = (6, 0)

path = a_star(matrix, start, target)

if path:
    print("Path found:", path)
else:
    print("No path found.")

Path found: [(0, 2), (4, 2), (4, 4), (6, 4), (6, 0)]
